<a href="https://colab.research.google.com/github/jcandane/CellList/blob/main/celllistboxbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/jcandane/CellList
!git clone https://github.com/jcandane/xyz_file_manager

Cloning into 'CellList'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
Cloning into 'xyz_file_manager'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from CellList.CellList import Cell_List
from xyz_file_manager.xyz_manager import xyz_reader

np.set_printoptions(precision=4, linewidth=200, threshold=2000, suppress=True)

def get_router(l):
    router  = np.triu( np.arange(0,l,1)[None,:] + np.cumsum( np.roll(np.flip(np.arange(0,l,1)), 1))[:,None] )
    router -= (np.eye(l, dtype=np.int64) - 1) * router.T
    return router

In [4]:
Z, R_ix = xyz_reader("Ar_1000.xyz")

IJ, indexes, sorted = Cell_List(R_ix, 14)

In [5]:
IJ.shape

(360, 2)

## Tabulated Potential

In [6]:
#### !!! We have to deal with 3 spatial units: Å (Ångström), a0 (Bohr), & u (tabulated units) !!!
## https://simonensemble.github.io/PorousMaterials.jl/dev/force_field/

## http://www.sklogwiki.org/SklogWiki/index.php/Argon
σ = 6.42507 ## in bohr xor 0.34 nm [Rahman 1964] 
#ε = 3.80e-4  ## 120 K  [Rahman 1964]
ε = np.array([3.80e-4, 5.00e-4, 3.00e-4]) ## np.linspace(1e-6, 1e-4, 100)

u      = 0.001 # Å
xi     = np.arange(0., 2.0, u)
x      = np.arange(2.0, 20, u) ### these are in Å, units, R_ix / u
LJE_Ar =  4*np.einsum("a, x -> ax", ε, ((σ/x)**12 - (σ/x)**6))
LJf_Ar = 24*np.einsum("a, x -> ax", ε, (σ**6)*(2*σ**6/(x**13) - 1/(x**7)))

LJE_Ari = np.zeros( (LJE_Ar.shape[0], xi.shape[0]) )
LJf_Ari = np.zeros( (LJf_Ar.shape[0], xi.shape[0]) )

LJE_Ar = np.concatenate((LJE_Ari, LJE_Ar), axis=1)
LJf_Ar = np.concatenate((LJf_Ari, LJf_Ar), axis=1)
E_nAR = np.asarray([LJE_Ar, LJf_Ar])



router = get_router(2)
print( router )

def LJ_tabulated(A, u=0.001):

    np.unique(A)
    pairtype = None

    return E_nAR, u, pairtype

np.unique(Z)

[[0 1]
 [1 2]]


array(['Ar'], dtype='<U32')

In [7]:
LJE_Ar =  4*np.einsum("a, x -> ax", ε, ((σ/x)**12 - (σ/x)**6))
LJE_Ari = np.zeros( (LJE_Ar.shape[0], xi.shape[0]) )

np.concatenate((LJE_Ari, LJE_Ar), axis=1)[:,2500]

#LJE_Ari.shape
E_nAR.shape

(2, 3, 20000)

## Box-Box Interaction

In [20]:
def calcBOXBOX(R_ix, AT_i,    IJ, indexes, sorted,     E_nAR, u, router):
    """
    GIVEN:  Atom Character;   R_ix (positions), AT_i (Unary Atom-Types)
            Box Neighborhood; IJ (Box Pairs), indexes (over all atoms), sorted (over all boxes)
            Potential;  E_nAR (Potential; n (derivative), A (pair-type), and R (Radial distance))
                        u (unit of potential vs Bohr)
                        router ()
    GET:    E (energy), f_ix (force), and g_r (pair-correlation)
    """

    R_ix *= u ## convert R_ix into tabulated units
    E     = 0.
    f_ix  = np.zeros(R_ix.shape)
    g_r   = np.zeros(E_nAR[0].shape)
    for i in range(len(IJ)):
        I, J  = IJ[i]
        I_i   = indexes[ sorted[I]:sorted[I+1] ] ## atom indices belonging to box I
        J_j   = indexes[ sorted[J]:sorted[J+1] ] ## atom indices belonging to box J

        ## displacement/distance calculation
        R_ijx    = R_ix[ I_i, None, : ] - R_ix[ None, J_j, : ]
        distance = np.linalg.norm(R_ijx, axis=2)
        R_ijx   *= 1/(distance[:,:,None] + 1e-10)
        N, dx    = distance.astype(int), distance-distance.astype(int)

        ## get pair-atom-types from unary-atom-types
        AT = router[ np.outer(AT_i[I_i], np.ones(len(J_j), dtype=int)) , np.outer(np.ones(len(I_i), dtype=int), AT_i[J_j]) ]

        ## save pair-correlation, pair-energy, & pair-forces
        g_r[AT, N] += 1
        E          += np.sum(E_nAR[ 0, AT, N ] + (E_nAR[ 0, AT, N+1 ] - E_nAR[ 0, AT, N ] ) * dx)
        f_ijx       =  (E_nAR[ 1, AT, N ] + (E_nAR[ 1, AT, N+1 ] - E_nAR[ 1, AT, N ] ) * dx)[:,:,None] * R_ijx
        f_ix[I_i]  += np.sum( f_ijx, axis=1)
        f_ix[J_j]  -= np.sum( f_ijx, axis=0)

    return E, f_ix, g_r 

start = time.time()

AT_i = np.zeros(Z.shape, dtype=int)

E, f_ix, g_r = calcBOXBOX(R_ix, AT_i,    IJ, indexes, sorted,     E_nAR, u, router)

print("net force = %4.12f a.u." %(np.sum(f_ix)))
print("timer     = %4.6f s" % (time.time()-start))

net force = 0.000000000000 a.u.
timer     = 0.371582 s


In [9]:
Element_Names = { 'e':0,  0:'e' ,   'H':  1,  1:'H' ,  'He':  2,  2:'He',
               'Li':  3,  3:'Li',  'Be':  4,  4:'Be',   'B':  5,  5:'B' ,   'C':  6,  6:'C' ,   'N':  7,  7:'N' ,   'O':  8,  8:'O' ,   'F':  9,  9:'F' ,  'Ne': 10, 10:'Ne', 
               'Na': 11, 11:'Na',  'Mg': 12, 12:'Mg',  'Al': 13, 13:'Al',  'Si': 14, 14:'Si',   'P': 15, 15:'P' ,   'S': 16, 16:'S' ,  'Cl': 17, 17:'Cl',  'Ar': 18, 18:'Ar',
                'K': 19, 19:'K' ,  'Ca': 20, 20:'Ca',  'Sc': 21, 21:'Sc',  'Ti': 22, 22:'Ti',   'V': 23, 23:'V' ,  'Cr': 24, 24:'Cr',  'Mn': 25, 25:'Mn',  'Fe': 26, 26:'Fe',  'Co': 27, 27:'Co',  'Ni': 28, 28:'Ni',  'Cu': 29, 29:'Cu',  'Zn': 30, 30:'Zn',  'Ga': 31, 31:'Ga',  'Ge': 32, 32:'Ge',  'As': 33, 33:'As',  'Se': 34, 34:'Se',  'Br': 35, 35:'Br',  'Kr': 36, 36:'Kr', 
               'Rb': 37, 37:'Rb',  'Sr': 38, 38:'Sr',   'Y': 39, 39:'Y' ,  'Zr': 40, 40:'Zr',  'Nb': 41, 41:'Nb',  'Mo': 42, 42:'Mo',  'Tc': 43, 43:'Tc',  'Ru': 44, 44:'Ru',  'Rh': 45, 45:'Rh',  'Pd': 46, 46:'Pd',  'Ag': 47, 47:'Ag',  'Cd': 48, 48:'Cd',  'In': 49, 49:'In',  'Sn': 50, 50:'Sn',  'Sb': 51, 51:'Sb',  'Te': 52, 52:'Te',   'I': 53, 53:'I' ,  'Xe': 54, 54:'Xe',
               'Cs': 55, 55:'Cs',  'Ba': 56, 56:'Ba',  'La': 57, 57:'La',  'Ce': 58, 58:'Ce',  'Pr': 59, 59:'Pr',  'Nd': 60, 60:'Nd',  'Pm': 61, 61:'Pm',  'Sm': 62, 62:'Sm',  'Eu': 63, 63:'Eu',  'Gd': 64, 64:'Gd',  'Tb': 65, 65:'Tb',  'Dy': 66, 66:'Dy',  'Ho': 67, 67:'Ho',  'Er': 68, 68:'Er',  'Tm': 69, 69:'Tm',  'Yb': 70, 70:'Yb',  'Lu': 71, 71:'Lu',  'Hf': 72, 72:'Hf',  'Ta': 73, 73:'Ta',   'W': 74, 74:'W' ,  'Re': 75, 75:'Re',  'Os': 76, 76:'Os',  'Ir': 77, 77:'Ir',  'Pt': 78, 78:'Pt',  'Au': 79, 79:'Au', 'Hg': 80, 80:'Hg',  'Tl': 81, 81:'Tl',  'Pb': 82, 82:'Pb',  'Bi': 83, 83:'Bi',  'Po': 84, 84:'Po', 'At': 85, 85:'At', 'Rn': 86, 86:'Rn', 
               'Fr': 87, 87:'Fr',  'Ra': 88, 88:'Ra',  'Ac': 89, 89:'Ac',  'Th': 90, 90:'Th',  'Pa': 91, 91:'Pa',   'U': 92, 92:'U' ,  'Np': 93, 93:'Np',  'Pu': 94, 94:'Pu',  'Am': 95, 95:'Am',  'Cm': 96, 96:'Cm',  'Bk': 97, 97:'Bk',  'Cf': 98, 98:'Cf',  'Es': 99, 99:'Es',  'Fm':100,100:'Fm',  'Md':101,101:'Md',  'No':102,102:'No',  'Lr':103,103:'Lr',  'Rf':104,104:'Rf',  'Db':105,105:'Db',  'Sg':106,106:'Sg',  'Bh':107,107:'Bh',  'Hs':108,108:'Hs',  'Mt':109,109:'Mt',  'Ds':110,110:'Ds',  'Rg':111,111:'Rg', 'Cn':112,112:'Cn',  'Nh':113,113:'Nh',  'Fl':114,114:'Fl',  'Mc':115,115:'Mc',  'Lv':116,116:'Lv', 'Ts':117,117:'Ts', 'Og':118,118:'Og'}

nums = np.arange(1,119,1, dtype=int)

for i in range(len(nums)):
    #print(Element_Names[i] + "\t")
    None

In [11]:
f = "SCHWERDTFEGER2018.txt"
fp = open(f, 'r')

lines = fp.readlines()
asd = []
qwe = []
for i in lines: #range(len(lines)):
    if (i)[0] != "[":
        if len(i.split()) > 2:
            if i.split()[3] == "recommended":
                qwe.append(i)
            else:
                asd.append(i.split()[1])
        else:
            continue

len(asd)

127

In [ ]:
qwe